### Ситуация: фильтрация спама вхоящей корреспонденции почтового ящика.
##### H0(гипотеза): данное, конкретное письмо - SPAM: т.е. Positive - маркер писма со спамом
- True_Positive  - письмо со спамом помечено как письмо-спам
- False_Positive - письмо со спамом помечено как письмо без спама (ошибка 1 рода) 
- False_Negative - письмо без спама помечено как письмо со спамом (ошибка 2 рода)
- True_Negative  - письмо без спама помечено как письмо без спама

- Precision (точность) - метрика, демонстрирующая способность алгоритма отличать конкретно рассматриваемый класс от других
- Precision = TP/(TP+FP)

- Recall (полнота) - метрика, демонстрирующая способность алгоритма онаруживать конкретно заданный класс
- Recall = TP/(TP+FN) < == > TP/Num_Positive
- Num_Positive = TP+FN (можно считать как количество объектов на картинке в эталонной разметке)  относительно одного класса

In [1]:
# https://towardsdatascience.com/map-mean-average-precision-might-confuse-you-5956f1bfa9e2

In [2]:
import numpy as np

In [3]:
# в нижеприведенных алгоритмах используется 2(из4-х)метрики: TP(TruePositive) и FP(FalsePositive)
# !ВАЖНО т.к. данные алгоритмы используются для задач детекции объектов, метрики TP и FP будут иметь накопительных 
# формат(1 изображение != 1 объект на картинке), соответственно, TP и FP будут просчитываться для каждого объекта 
# на изображении.

# основыне данные - матрица соответствия Precision/Recall

In [3]:
def calculate_ap_every_point(rec, prec):
    mrecall    = [0]; [mrecall.append(e)    for e in  rec]; mrecall.append(1)
    mprecision = [0]; [mprecision.append(e) for e in prec]; mprecision.append(0)
    
    for i in range(len(mprecision) - 1, 0, -1):
        mprecision[i - 1] = max(mprecision[i - 1], mprecision[i])
    
    ii = []
    for i in range(len(mrecall) - 1):
        if mrecall[1:][i] != mrecall[0:-1][i]:
            ii.append(i + 1)
    
    # подобие ручного подсчета интеграла: 
    # mrecall[i] - mrecall[i - 1] - ширина столбца на графике precision/recall
    # mprecision[i] - высота столбца на графике precision/recall
    ap = 0
    for i in ii:
        ap = ap + np.sum((mrecall[i] - mrecall[i - 1]) * mprecision[i])
        
    return [ap, mprecision[0:len(mprecision) - 1], mrecall[0:len(mprecision) - 1], ii]

In [4]:
recall = [0.33333333, 0.66666667, 1.]
precision = [1., 1., 1.]
r = calculate_ap_every_point(recall, precision)
r

[1.0, [1.0, 1.0, 1.0, 1.0], [0, 0.33333333, 0.66666667, 1.0], [1, 2, 3]]

In [5]:
# для случая подсчета AP на 11 точках:
# 
def calculate_ap_11_point_interp(recall, precision, recall_vals=11):
    mrec         = [e for e in recall]
    mpre         = [e for e in precision]
    rhoInterp    = []
    recallValid  = []
    # array([1.0, 0.9, 0.8, 0.7, 0.6, 0.5, 0.4, 0.3, 0.2, 0.1, 0.0])
    recallValues = np.linspace(1, 0, recall_vals)
    
    for r in recallValues:
        # ищет индексы из списка recall, значения которого больше порога r : 1.0, 0.9, 0.8 ...
        argGreaterRecalls = np.argwhere(mrec[:] >= r)
        pmax = 0
        if argGreaterRecalls.size != 0:
            # Если индексы имеются - выбирается минимальный индекс, по этому индексу берется 
            # слайс из precision, из слайса выбирается максимвльный по значению показатель
            
            # ---------------------------------------------------------
            # !!! Вопрос по красным точкам решается тут
            # ---------------------------------------------------------
            pmax = max(mpre[argGreaterRecalls.min():])
            
        recallValid.append(r)
        rhoInterp.append(pmax)
    # By definition AP = sum(max(precision whose recall is above r))/11
    ap = sum(rhoInterp) / len(recallValues)
    
    # Generating values for the plot
    rvals = [recallValid[0]]; [rvals.append(e) for e in recallValid]; rvals.append(0)
    pvals =              [0]; [pvals.append(e) for e in   rhoInterp]; pvals.append(0)
    cc = []
    for i in range(len(rvals)):
        p = (rvals[i], pvals[i - 1])
        if p not in cc:
            cc.append(p)
        p = (rvals[i], pvals[i])
        if p not in cc:
            cc.append(p)
    recallValues = [i[0] for i in cc]
    rhoInterp = [i[1] for i in cc]
    
    return [ap, rhoInterp, recallValues, None]

In [6]:
recall = [0.33333333, 0.66666667, 1.]
precision = [1., 1., 1.]
r = calculate_ap_11_point_interp(recall, precision, recall_vals=11)
r

[1.0,
 [0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 0],
 [1.0,
  1.0,
  0.9,
  0.8,
  0.7,
  0.6,
  0.5,
  0.3999999999999999,
  0.29999999999999993,
  0.19999999999999996,
  0.09999999999999998,
  0.0,
  0],
 None]